In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt 

In [70]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\HDFC\HDFC\hdfc01.pdf'

In [71]:
pars = tabula.read_pdf(f,                                            
                       pages='all',
                       silent=True,
                       stream=True,
                       multiple_tables = True,
                       pandas_options={ 'header': None }) 

In [72]:
pars

[           0                                         1                 2  \
 0       Date                                 Narration      Chq./Ref.No.   
 1   01/02/18   IMPS-803208815874-RAVI KUMAR-HDFC-XXXXX  0000803208815874   
 2        NaN                            X1767-COMMENTS               NaN   
 3   01/02/18                        CASH DEP KARAMPURA   000000000000000   
 4   01/02/18  ACH D- LENDING KART FINANCE-LK0000016990  0000001677048417   
 5   01/02/18  ACH D- NEOGROWTH CREDIT PVT-1017745_ C45  0000001687879449   
 6   01/02/18  ACH D- NEOGROWTH CREDIT PVT-1017745_ B42  0000001687602395   
 7   01/02/18  ACH D- UNITED PETRO FINANCE-100610100127  0000001688762449   
 8   01/02/18  IMPS-803219133729-PRADEEP KUMAR-HDFC-XXX  0000803219133729   
 9        NaN                     XXXXX0307-1113961428/               NaN   
 10  01/02/18  IMPS-803219015861-PRADEEP KUMAR-HDFC-XXX  0000803219015861   
 11       NaN                     XXXXX0307-1113961431/               NaN   

In [10]:
df = pd.DataFrame()

In [11]:
tables = []
for i,c in enumerate(pars):
    if c.shape[1] == 7:
        tables.append(c)
    elif c.shape[1] == 6:
        c[6]=np.nan; c = c[[0,1,2,3,4,6,5]]; c.columns=range(c.shape[1]);
        tables.append(c)
    elif c.shape[1] == 8:
        c = c[[0,1,3,4,5,6,7]];c.columns = range(c.shape[1]);
        tables.append(c)
        
df = pd.concat(tables, ignore_index=True)

df.to_csv("check01.csv", index=0)

In [25]:
try:
    idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
        except:
            print("\nHDFC-Column headers missing"); log.write("\nHDFC-Column headers missing"); pass
        
try:
    idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('statement summary').any(),axis=1)==True].index][0]    
    df.drop(df.index[idx2:], inplace=True)
except:pass

In [26]:
# df.to_csv("check02.csv", index=0)

In [27]:
df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])

In [28]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
    df[bal]=df[bal].apply( lambda x: abc(x) )
except: print("\nBalance columns missing")           

df['flag']=df.iloc[:,-1].shift(1)
df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]
 
try:
    wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in df.columns if "AMOUNT" in str(c).upper() ][0]
    except: pass   

try:
    dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
except: pass             

try:
    df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
    df['Wdl1']=df[wdl].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1
    df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) )                
except: 
    try:
        Wdl1=[]
        for i, item in enumerate(df[wdl]):
            try:
                tmpp=float(str(item).replace(",","").replace("(Cr)","").replace("(Dr)","")) *-1
            except ValueError:
                df.iloc[i,df.columns.get_loc(wdl)-1]=df.iloc[i,df.columns.get_loc(wdl)]
                df.iloc[i,df.columns.get_loc(wdl)]=df.iloc[i,df.columns.get_loc(dep)]
                df.iloc[i,df.columns.get_loc(dep)]=df.iloc[i,df.columns.get_loc(bal)]
                df.iloc[i,df.columns.get_loc(bal)]=df.iloc[i,df.columns.get_loc(bal)+1]
            Wdl1.append(tmpp)  
        df[bal]=df[bal].fillna(df.iloc[:,df.columns.get_loc(bal)-1])
        df.drop(df.columns[df.columns.get_loc(bal)+1],axis=1,inplace=True)
        df['Wdl1']=Wdl1   
        df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) )                
    except:pass

In [29]:
df=df.T.drop_duplicates().T           
df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
df['flag']=df.iloc[:,0].astype(str)+df['Wdl1'].astype(str) +df[bal].astype(str)
df['flag2']=np.arange(len(df))

df.loc[ df[['flag']].duplicated(keep=False) , 'flag' ] =df['flag'] + df['flag2'].astype(str)  
df['flag']=df['flag'].apply( lambda row : np.nan if 'nannannan' in row else row ).fillna(method='ffill')        

df['Narration']=df.groupby('flag')['Narration'].transform( lambda x :' '.join(x))                
df=df.drop_duplicates(['flag'],keep='first').iloc[0:,:-3].reset_index(drop=True)

In [30]:
# df.to_csv("check03.csv", index=0)

In [31]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [32]:
df[dep][0]

'20,000.00'

In [33]:
def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False

if isnan(df[dep][0]) == False:
    df["Credits"] = np.nan
    df["Credits"][0] = float(str(df[dep][0]).replace(",","").replace("(Cr)","").replace("(Dr)","")) 
    df["Debits"] = np.nan
elif isnan(df[wdl][0]) == False:
    df["Credits"] = np.nan    
    df["Debits"] = np.nan
    df["Debits"][0] = float(str(df[wdl][0]).replace(",","").replace("(Cr)","").replace("(Dr)","")) *-1  

c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [34]:
df.head()

,Date,Narration,Chq./Ref.No.,Withdrawal Amt.,Deposit Amt.,Closing Balance,Credits,Debits
0,01/11/19,CA0005937660SHIVESH MOBILE,0000000000036780,NaN,"20,000.00",20000.0,20000.0,NaN
1,06/11/19,EDC ANNUAL RENTAL 2019 LA3949,000000000000000,"4,720.00",NaN,15280.0,NaN,NaN
2,06/11/19,41640146TERMINAL 1 CARDS SETTL. 26/10/19,000000000000000,NaN,101.00,15381.0,NaN,NaN
3,11/11/19,CASH DEP MIRA ROAD BE,000000000000000,NaN,"12,500.00",27881.0,NaN,NaN
4,13/11/19,41640146TERMINAL 1 CARDS SETTL. 13/11/19,000000000000000,NaN,"68,967.50",96848.5,NaN,NaN


In [35]:
# df2 = df

In [36]:
for i,j in enumerate(df[bal]):
    if i < len(df[bal])-1 :
        if df[bal][i] < df[bal][i+1]:
            df["Credits"][i+1] = df[bal][i+1]-df[bal][i]
        elif df[bal][i] > df[bal][i+1]:
            df["Debits"][i+1] = df[bal][i+1]-df[bal][i]
    else:
        pass

c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [37]:
# df.to_excel("check04.xlsx", index=0)

In [38]:
df = df[["Date","Chq./Ref.No.","Narration","Debits","Credits","Closing Balance"]]
df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]

In [39]:
df.to_excel("hdfc01.xlsx", index=False)

In [ ]:
tables = []
for i,c in enumerate(pars):
    if i == (len(pars)-1):
        if c.shape[1] == 7:
            tables.append(c)
        elif c.shape[1] == 6:
            c[6]=np.nan; c = c[[0,1,2,3,4,6,5]]; c.columns=range(c.shape[1]);
            tables.append(c)
        elif c.shape[1] == 8:
            c = c[[0,1,3,4,5,6,7]];c.columns = range(c.shape[1]);
            tables.append(c)
    else:
        if c.shape[1] == 7:
            tables.append(c)
        elif c.shape[1] == 6:
            tables.append(c)
        elif c.shape[1] == 8:
            c = c[[0,1,3,4,5,6,7]];c.columns = range(c.shape[1]);
            tables.append(c)

In [ ]:
print(len(pars)-1)

In [ ]:
df = pd.concat(tables, ignore_index=True)

df.to_csv("check.csv", index=0)

In [ ]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\HDFC\files\hdfc11.pdf'

pars = tabula.read_pdf(f,                                            
                       pages='all',
                       silent=True,
                       stream=True,
                       multiple_tables = True,
                       pandas_options={ 'header': None }) 

df = pd.DataFrame()

In [ ]:
df = pd.concat([c for c in pars]).drop_duplicates()

In [ ]:
df.to_csv("check.csv", index=0)

In [ ]:
def abc(a):
    if type(a) == str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else:
        z=a
    return z

pars = tabula.read_pdf(f,                                            
                       pages='all',
                       silent=True,
                       multiple_tables = True,
                       pandas_options={ 'header': None }  )          

for k in [ c for c in pars if c.shape[1]>=6 and c.shape[0]>2]:
    k.dropna(how='all',axis=1,inplace=True);   k.columns=list(range(k.shape[1]))    

df=pd.DataFrame()  
try:
    df=pd.concat([ c for c in pars if c.shape[1]>=6 and c.shape[0]>2 and c.iloc[0,0] !='Type of Account']).drop_duplicates() 
except: pass
try:
    df=pd.concat([ c for c in pars if c.shape[1]==9 and c.shape[0]==1 and c.iloc[0,0] !='Type of Account']).drop_duplicates() 
except: pass    
try:
    idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
        except:
            print("\nHDFC-Column headers missing"); log.write("\nHDFC-Column headers missing"); pass

df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)             
df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
    df[bal]=df[bal].apply( lambda x: abc(x) )
except: print("\nBalance columns missing")           

df['flag']=df.iloc[:,-1].shift(1)
df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

try:
    wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in df.columns if "AMOUNT" in str(c).upper() ][0]
    except: pass   

try:
    dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
except: pass             

try:
    df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
    df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )                       
    df['Wdl1']=df[wdl].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1
    df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) )                
except: 
    try:
        Wdl1=[]
        for i, item in enumerate(df[wdl]):
            try:
                tmpp=float(str(item).replace(",","").replace("(Cr)","").replace("(Dr)","")) *-1
            except ValueError:
                df.iloc[i,df.columns.get_loc(wdl)-1]=df.iloc[i,df.columns.get_loc(wdl)]
                df.iloc[i,df.columns.get_loc(wdl)]=df.iloc[i,df.columns.get_loc(dep)]
                df.iloc[i,df.columns.get_loc(dep)]=df.iloc[i,df.columns.get_loc(bal)]
                df.iloc[i,df.columns.get_loc(bal)]=df.iloc[i,df.columns.get_loc(bal)+1]
            Wdl1.append(tmpp)  
        df[bal]=df[bal].fillna(df.iloc[:,df.columns.get_loc(bal)-1])
        df.drop(df.columns[df.columns.get_loc(bal)+1],axis=1,inplace=True)
        df['Wdl1']=Wdl1   
        df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) )                
    except:pass            

df=df.T.drop_duplicates().T           
df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
df['flag']=df.iloc[:,0].astype(str)+df['Wdl1'].astype(str) +df[bal].astype(str)
df['flag2']=np.arange(len(df))

df.loc[ df[['flag']].duplicated(keep=False) , 'flag' ] =df['flag'] + df['flag2'].astype(str)  
df['flag']=df['flag'].apply( lambda row : np.nan if 'nannannan' in row else row ).fillna(method='ffill') 

print(df.tail())

df['Narration']=df.groupby('flag')['Narration'].transform( lambda x :''.join(x))                
df=df.drop_duplicates(['flag'],keep='first').iloc[0:,:-3].reset_index(drop=True)

In [ ]:
df.tail()